In [2]:
!pip uninstall google-generativeai google-ai-generativelanguage langchain-google-genai -y
!pip install -qU google-generativeai langchain-google-genai langchain-pinecone

Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
Found existing installation: google-ai-generativelanguage 0.6.15
Uninstalling google-ai-generativelanguage-0.6.15:
  Successfully uninstalled google-ai-generativelanguage-0.6.15
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [24]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


In [ ]:
# ✅ Set your API Keys
import os
os.environ['GOOGLE_API_KEY'] = 'qewr'
os.environ['PINECONE_API_KEY'] = 'dsfa'

In [5]:
# ✅ Load CSV and Convert to Documents
import pandas as pd
from langchain.schema import Document

df = pd.read_csv('/content/dummy_products_extended.csv')
documents = []
for _, row in df.iterrows():
    content = f"{row['title']}\n{row['description']}\nPrice: ${row['price']}"
    metadata = {
        'category': row['category'],
        'brand': row['brand'],
        'price': float(row['price']),
        'source': 'product_catalog'
    }
    documents.append(Document(page_content=content, metadata=metadata))

In [33]:
print(documents)

[Document(metadata={'category': 'Wearables', 'brand': 'FitTech', 'price': 162.2, 'source': 'product_catalog'}, page_content='Waterproof Smartwatch v3.3\nStay connected while swimming or running. This smartwatch is waterproof and includes heart rate monitoring. Now with improved performance version 1.3.\nPrice: $162.2'), Document(metadata={'category': 'Audio', 'brand': 'SoundPro', 'price': 73.4, 'source': 'product_catalog'}, page_content='Wireless Bluetooth Headphones v3.9\nHigh-quality wireless headphones with noise isolation and 20-hour battery life. Now with improved performance version 1.6.\nPrice: $73.4'), Document(metadata={'category': 'Footwear', 'brand': 'RunFast', 'price': 76.78, 'source': 'product_catalog'}, page_content="Running Shoes - Men's v5.1\nLightweight and breathable shoes designed for runners. Offers extra heel support. Now with improved performance version 1.7.\nPrice: $76.78"), Document(metadata={'category': 'Electronics', 'brand': 'VisionX', 'price': 500.22, 'sour

In [6]:
# ✅ Split documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
docs = splitter.split_documents(documents)

In [34]:
print(docs)

[Document(metadata={'category': 'Wearables', 'brand': 'FitTech', 'price': 162.2, 'source': 'product_catalog', 'text': 'Waterproof Smartwatch v3.3\nStay connected while swimming or running. This smartwatch is waterproof and includes heart rate monitoring. Now with improved performance version 1.3.\nPrice: $162.2'}, page_content='Waterproof Smartwatch v3.3\nStay connected while swimming or running. This smartwatch is waterproof and includes heart rate monitoring. Now with improved performance version 1.3.\nPrice: $162.2'), Document(metadata={'category': 'Audio', 'brand': 'SoundPro', 'price': 73.4, 'source': 'product_catalog', 'text': 'Wireless Bluetooth Headphones v3.9\nHigh-quality wireless headphones with noise isolation and 20-hour battery life. Now with improved performance version 1.6.\nPrice: $73.4'}, page_content='Wireless Bluetooth Headphones v3.9\nHigh-quality wireless headphones with noise isolation and 20-hour battery life. Now with improved performance version 1.6.\nPrice: $7

In [7]:
# ✅ Embed with Gemini Embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [9]:

from pinecone import Pinecone, ServerlessSpec
import time

index_name = 'ecom-rag-index'
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])



pc.create_index(
    name=index_name,
    dimension=3072,
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1'),
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'e27bfdeb74dfc46171e2f1adf86f0d51', 'date': 'Sat, 02 Aug 2025 15:28:35 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [10]:
index = pc.Index(index_name)

In [11]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [12]:
# ✅ Upload vectors
from uuid import uuid4

from langchain_core.documents import Document

uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=docs, ids=uuids)

['54e0f18b-dd51-46ae-82c9-4752c6bc5acb',
 'f0c393b8-d525-422d-ab4b-8c4803765a1a',
 '624eb0cb-971e-4926-ab56-65dfa0a9fec2',
 '4153700c-dae1-4794-bae2-caa40b42923c',
 '04bc7cbb-6e98-4c2c-b4a6-176c96109ddc',
 '103d2895-fad1-4f38-b1c3-f40c0b5422f3',
 'c1dc72a9-6d80-44e8-8a60-9b2a2ae314d1',
 '79655fce-c021-42bc-86d9-9dfbdbe1f63e',
 'cfc9dfc2-2fc9-4a38-b618-cc404103b23f',
 '68444a70-8fcb-4464-97ab-38ee963eb7a4',
 'aa63c39e-208d-42a9-9d4d-d22c729afcba',
 '83f75ee8-1d83-485f-8df8-6aefc79857e4',
 'b60e80e5-dcd0-4e7f-b75e-d98ab2089bdb',
 '8701fb09-65e1-42d4-ab14-ad3f8b4564ec',
 'dd52c923-728e-4060-8b8f-fd5fa78553df',
 '2b4b0144-3fad-47cb-813d-ae88590e0f3d',
 'd955bd53-135a-474a-9632-e4bf2140bc0c',
 'e5b08645-47d4-4c0f-a1b7-2db784456e68',
 'dd591ba7-aa5a-4ea5-9a58-ed88807c6887',
 'd734f158-4cba-4e84-850d-43a76ee3f713',
 'f82a2ae7-46a8-4f8e-8d08-2b8a9695eb48',
 '9800ae04-cc32-4ac9-aeea-d0d9b3b18218',
 'f5899427-bf51-40e5-8f48-9599fde44ce3',
 '759bd235-40f5-42ac-8193-af2281c38b70',
 '6cc54fd8-a0ec-

In [31]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash', temperature=0.2)
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever()
)

In [32]:
# ✅ Ask a question
query = 'What are the best waterproof smartwatches under $200?'
answer = rag_chain.run(query)
print('💬', answer)

💬 Based on the information provided, here are the waterproof smartwatches under $200:

*   **Waterproof Smartwatch v3.3:**
    *   Price: $162.2
    *   Features: Waterproof, heart rate monitoring, improved performance version 1.3.

*   **Waterproof Smartwatch v2.4:**
    *   Price: $136.64
    *   Features: Waterproof, heart rate monitoring, improved performance version 5.9.
